In [6]:
import pandas as pd
import os

output_domain_size_dict = {
    "smartgrid-1": (3, 12),
    "smartgrid-2": (9, 12),
    "prob-termination-5": (6, 10),
    "prob-termination-7": (8, 10),
    "prob-termination-6": (7, 10),
    "prob-termination-9": (10, 10),
    "prob-termination-12": (13, 20),
    "window-16": (16, 16),
    "smartgrid-3": (27, 12),
    "reservoir-6": (64, 8),
    "window-20": (20, 20),
    "window-24": (24, 24),
    "window-28": (28, 28),
    "window-32": (32, 32),
    "smartgrid-4": (81, 12),
    "smartgrid-5": (243, 12),
    "reservoir-8": (256, 16),
    "random-walk-14": (500, 40),
    "random-walk-3": (500, 24),
    "random-walk-5": (500, 31),
    "random-walk-7": (500, 33),
    "reservoir-10": (1024, 32),
}

method_rename = {
    "empirical": "Empirical",
    "miller": "Miller",
    "HyLeak": "HyLeak",
    "ChaoFON": "ChaoFON",
    "ChaoFRN": "ChaoFRN",
    "ChaoION": "ChaoSON",
    "ChaoIRN": "ChaoSRN",
    "ChaoFOM": "ChaoFOM",
    "ChaoFRM": "ChaoFRM",
    "ChaoIOM": "ChaoSOM",
    "ChaoIRM": "ChaoSRM",
}
method_order = [
    "Empirical",
    "Miller",
    "HyLeak",
    "ChaoFON",
    "ChaoFRN",
    "ChaoSON",
    "ChaoSRN",
    "ChaoFOM",
    "ChaoFRM",
    "ChaoSOM",
    "ChaoSRM",
]


In [7]:
df = pd.DataFrame(
    columns=["subject", "GT", "ratio", "method", "MI", "SE", "o.e."]
)
for subject in output_domain_size_dict.keys():
    data_path = f"../result/esti-merged-{subject}-i-xy.csv"
    if os.path.exists(data_path) == False:
        continue
    data = pd.read_csv(data_path)
    GT = data["GT"].values[0]
    data = data.drop(columns=["GT"])
    data_melt = data.melt(
        id_vars=["Nx", "trial"], var_name="method", value_name="MI"
    )
    # method rename
    data_melt["method"] = data_melt["method"].map(method_rename)
    data_melt["method"] = pd.Categorical(data_melt["method"], method_order)
    data_melt["SE"] = (data_melt["MI"] - GT) ** 2
    data_mean = (
        data_melt.drop(columns=["SE"])
        .groupby(["Nx", "method"], observed=False)
        .mean()
        .reset_index()
        .drop(columns=["trial"])
    )
    data_mse = (
        data_melt.drop(columns=["MI"])
        .groupby(["Nx", "method"], observed=False)
        .mean()
        .reset_index()
        .drop(columns=["trial"])
    )
    data_mean = data_mean.merge(data_mse, on=["Nx", "method"])
    NS_x, NS_y = output_domain_size_dict[subject]
    data_mean["ratio"] = data_mean["Nx"] / NS_y
    data_mean.loc[data_mean["ratio"] < 1, "ratio"] = 0.5
    data_mean = data_mean.drop(columns=["Nx"])
    data_mean["subject"] = subject
    data_mean["GT"] = GT
    data_mean["o.e."] = data_mean["MI"] - GT > 0
    df = pd.concat([df, data_mean], ignore_index=True)
df = df.dropna(subset=["MI"])
display(df)


/var/folders/v9/wc5dh1fd691947y71hyt_2280000gn/T/ipykernel_19588/829916277.py:40: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  df = pd.concat([df, data_mean], ignore_index=True)
/var/folders/v9/wc5dh1fd691947y71hyt_2280000gn/T/ipykernel_19588/829916277.py:40: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  df = pd.concat([df, data_mean], ignore_index=True)
/var/folders/v9/wc5dh1fd691947y71hyt_2280000gn/T/ipykernel_19588/829916277.py:40: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  df = pd.concat([df, data_mean], ignore_index=True)
/var/folders/v9/wc5dh1fd691947y71hyt_2280000gn/T/ipykernel_19588/829916277.p

,subject,GT,ratio,method,MI,SE,o.e.
0,smartgrid-1,0.065825,0.5,Empirical,0.394855,0.127088,True
1,smartgrid-1,0.065825,0.5,Miller,0.198559,0.030786,True
2,smartgrid-1,0.065825,0.5,HyLeak,0.113302,0.013150,True
3,smartgrid-1,0.065825,0.5,ChaoFON,0.386327,0.127033,True
4,smartgrid-1,0.065825,0.5,ChaoFRN,0.395682,0.128371,True
...,...,...,...,...,...,...,...
963,reservoir-10,1.295879,5.0,ChaoSRN,1.319688,0.000582,True
964,reservoir-10,1.295879,5.0,ChaoFOM,1.384964,0.007971,True
965,reservoir-10,1.295879,5.0,ChaoFRM,1.518988,0.049798,True
966,reservoir-10,1.295879,5.0,ChaoSOM,1.319697,0.000582,True


In [8]:
def compare(method1: str, method2: str, metric: str, ratio=None, debug=False):
    sub_df = df.copy().reset_index(drop=True)
    if ratio is not None:
        sub_df = sub_df[sub_df["ratio"].isin(ratio)]
    method1_df = sub_df[sub_df["method"] == method1].set_index(
        ["subject", "ratio"]
    )
    if method2 is not None:
        method2_df = sub_df[sub_df["method"] == method2].set_index(
            ["subject", "ratio"]
        )
    if debug and method2 is not None:
        pd.set_option("display.max_rows", None)
        display(method1_df[metric] - method2_df[metric])
        pd.reset_option("display.max_rows")
    if method2 is not None:
        diff = (method1_df[metric] - method2_df[metric]) > 0
        zero = (method1_df[metric] - method2_df[metric]) == 0
    else:
        diff = method1_df[metric] > 0
        zero = method1_df[metric] == 0
    return (diff.sum(), zero.sum(), len(diff), diff.sum() / len(diff))


def diff(method1: str, method2: str, metric: str, ratio=None, debug=False):
    sub_df = df.copy().reset_index(drop=True)
    if ratio is not None:
        sub_df = sub_df[sub_df["ratio"].isin(ratio)]
    method1_df = sub_df[sub_df["method"] == method1].set_index(
        ["subject", "ratio"]
    )
    if method2 is not None:
        method2_df = sub_df[sub_df["method"] == method2].set_index(
            ["subject", "ratio"]
        )
    if debug and method2 is not None:
        pd.set_option("display.max_rows", None)
        display(method1_df[metric] - method2_df[metric])
        pd.reset_option("display.max_rows")
    if method2 is not None:
        diff = (method1_df[metric] - method2_df[metric])
    else:
        diff = method1_df[metric]
    return diff.mean(), diff.std()


def relative_error(method, ratio=None, only_negative=False):
    sub_df = df.copy().reset_index(drop=True)
    if ratio is not None:
        sub_df = sub_df[sub_df["ratio"].isin(ratio)]
    method_df = sub_df[sub_df["method"] == method].set_index(
        ["subject", "ratio"]
    )
    return (
        (method_df["MI"] - method_df["GT"])
        * ((1 - method_df["o.e."]) if only_negative else 1)
        / method_df["GT"]
    ).mean()

# Empirical vs Chao

In [12]:
print("Empirical > ChaoFON:", compare("Empirical", "ChaoFON", "MI"))
print("Empirical > ChaoFRN:", compare("Empirical", "ChaoFRN", "MI"))
print("Empirical > ChaoSON:", compare("Empirical", "ChaoSON", "MI"))
print("Empirical > ChaoSRN:", compare("Empirical", "ChaoSRN", "MI"))

Empirical > ChaoFON: (29, 1, 88, 0.32954545454545453)
Empirical > ChaoFRN: (57, 1, 88, 0.6477272727272727)
Empirical > ChaoSON: (69, 0, 88, 0.7840909090909091)
Empirical > ChaoSRN: (88, 0, 88, 1.0)


In [13]:
print("Empirical > ChaoFON:", compare("Empirical", "ChaoFON", "SE"))
print("Empirical > ChaoFRN:", compare("Empirical", "ChaoFRN", "SE"))
print("Empirical > ChaoSON:", compare("Empirical", "ChaoSON", "SE"))
print("Empirical > ChaoSRN:", compare("Empirical", "ChaoSRN", "SE"))

Empirical > ChaoFON: (26, 0, 88, 0.29545454545454547)
Empirical > ChaoFRN: (57, 0, 88, 0.6477272727272727)
Empirical > ChaoSON: (69, 0, 88, 0.7840909090909091)
Empirical > ChaoSRN: (88, 0, 88, 1.0)


# Flatten vs ByInput

In [14]:
print("ChaoFON > ChaoSON:", compare("ChaoFON", "ChaoSON", "MI"))
print("ChaoFRN > ChaoSRN:", compare("ChaoFRN", "ChaoSRN", "MI"))

ChaoFON > ChaoSON: (67, 0, 88, 0.7613636363636364)
ChaoFRN > ChaoSRN: (43, 0, 88, 0.48863636363636365)


In [15]:
print("ChaoFON > ChaoSON:", compare("ChaoFON", "ChaoSON", "SE"))
print("ChaoFRN > ChaoSRN:", compare("ChaoFRN", "ChaoSRN", "SE"))

ChaoFON > ChaoSON: (68, 0, 88, 0.7727272727272727)
ChaoFRN > ChaoSRN: (47, 0, 88, 0.5340909090909091)


# Refinement

In [16]:
print("ChaoFON > ChaoFRN:", compare("ChaoFON", "ChaoFRN", "MI"))
print("ChaoSON > ChaoSRN:", compare("ChaoSON", "ChaoSRN", "MI"))

ChaoFON > ChaoFRN: (61, 1, 88, 0.6931818181818182)
ChaoSON > ChaoSRN: (67, 1, 88, 0.7613636363636364)


In [17]:
print("ChaoFON > ChaoFRN:", compare("ChaoFON", "ChaoFRN", "SE"))
print("ChaoSON > ChaoSRN:", compare("ChaoSON", "ChaoSRN", "SE"))

ChaoFON > ChaoFRN: (60, 1, 88, 0.6818181818181818)
ChaoSON > ChaoSRN: (67, 1, 88, 0.7613636363636364)


# Müller's bias correction

In [20]:
print("Empirical > Miller:", compare("Empirical", "Miller", "SE"))
print("ChaoFRN > ChaoFRM:", compare("ChaoFRN", "ChaoFRM", "SE"))
print("ChaoSRN > ChaoSRM:", compare("ChaoSRN", "ChaoSRM", "SE"))

Empirical > Miller: (72, 0, 88, 0.8181818181818182)
ChaoFRN > ChaoFRM: (54, 0, 88, 0.6136363636363636)
ChaoSRN > ChaoSRM: (82, 0, 88, 0.9318181818181818)


In [22]:
print(
    compare("Empirical", None, "o.e.")[0] - compare("Miller", None, "o.e.")[0]
)
print(compare("ChaoFRN", None, "o.e.")[0] - compare("ChaoFRM", None, "o.e.")[0])
print(compare("ChaoSRN", None, "o.e.")[0] - compare("ChaoSRM", None, "o.e.")[0])


47
55
6


How much is the difference

In [23]:
print("Empirical > Miller:", diff("Empirical", "Miller", "MI"))
print("ChaoFRN > ChaoFRM:", diff("ChaoFRN", "ChaoFRM", "MI"))
print("ChaoSRN > ChaoSRM:", diff("ChaoSRN", "ChaoSRM", "MI"))

Empirical > Miller: (0.3483490324124308, 0.27903778827334974)
ChaoFRN > ChaoFRM: (0.2322538569082875, 0.21990372993622775)
ChaoSRN > ChaoSRM: (0.051143222653401876, 0.046709385659958205)
